<br>

# `#01 Simple Server Client:`

<br>

In [ ]:

# === client.py ===

import socket
client = socket.socket()
client.connect(('localhost',9999))
# take a name from clinet:
name = input("Enter your name: ")
client.send(bytes(f'{name}','utf-8'))

# now set the buffersize and print what client sending:
print(client.recv(2048))

# decode for printing in string:
print(client.recv(2048).decode())



#server.py

import socket

server = socket.socket()
server.bind(("localhost",9999))

# max 3 client:
server.listen(3)
print("Waiting for Connection....")

while True:
    client_socket,client_addr =  server.accept()
    # received from client:
    name = client_socket.recv(2048).decode()
    
    print(f"Connected with address: {client_addr}, client name: {name}")
    
    client_socket.send(bytes("Hi! from Yasin's Server","utf-8"))
    client_socket.close()



<br>

# `#02 FTP Server:`

<br>

In [ ]:

# === tcp ftp client.py ===

import socket

client = socket.socket()
client.connect(('localhost',9999))
client.settimeout(5)

filename = input("Enter the file name to send: ")

with open(filename,'rb') as f:
    while True:
        chunk = f.read(100)
        if not chunk:
            break
        ask_received = False
        while not ask_received:
            try:
                client.send(chunk)
                print("send successfully the chunk")
                ack = client.recv(1024)
                if ack==b"ACK":
                    print("ack received.")
                    ask_received=True
            except socket.timeout:
                print("Timeout")
print("file transfer complete")
client.close()
         
# === tcp_ftp_server.py ===

import socket
server = socket.socket()
server.bind(('localhost',9999))
server.listen(3)
print("server is wating for establish connection .....")

while True:
   client_socket,client_address =  server.accept()
   print(f"received client: {client_socket}")
   with open('received_file.txt','wb') as f:
       data = client_socket.recv(1024)
       if not data:
           break 
       f.write(data)
       client_socket.send(b"ACK")
       print(f"received successfully, send ACK")
       
   print("file transfer complete")
   client_socket.close()
server.close()


# === udp_ftp_client.py ===

import socket

client = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
filename = input("Enter the filename: ")

with open(filename,'r') as f:
    for line in f:
        client.sendto(line.encode(),('localhost',9999))
client.sendto(b'EOF',('localhost',9999))
print("file transfer complete")
client.close()


# === udp_ftp_server.py ===

import socket
server = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
server.bind(('localhost',9999))
print("udp server is listening ....")

with open("received_file_udp.txt",'w') as f:
    while True:
        data, addr = server.recvfrom(1024)
        if data==b'EOF':
            print("file transfer complete")
            break
        line = data.decode()
        f.write(line)
        print("received")
server.close()




<br>

# `#03 Concurrent_file_transfer:`

<br>

In [ ]:


# === client.py ===

import socket

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('localhost', 9999))

filename = input("Enter filename to download: ")
client.send(filename.encode())
response = client.recv(1024)

if response == b'OK':
    with open(f'downloaded_{filename}', 'wb') as f:
        while True:
            data = client.recv(1000)
            if not data:
                break
            f.write(data)
    print(f"File downloaded as downloaded_{filename}")
else:
    print(response.decode())
client.close()


# === server.py ===
import socket
import threading
import time
import os

def handle_client(conn, addr):
    print(f"Thread started for client {addr}")
    
    filename = conn.recv(1024).decode()
    print(f"Client {addr} requested: {filename}")
    
    if os.path.exists(filename):
        conn.send(b'OK')
        with open(filename, 'rb') as f:
            while True:
                data = f.read(1000)
                if not data:
                    break
                conn.send(data)
                time.sleep(0.2)
        print(f"File {filename} sent to {addr}")
    else:
        conn.send(b'ERROR: File not found')
    
    conn.close()

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(("localhost", 9999))
server.listen(5)

print(f"Server listening on {'localhost'}:{9999}")

while True:
    conn, addr = server.accept()
    print(f"Connection from {addr}")
    thread = threading.Thread(target=handle_client, args=(conn, addr))
    thread.start()
    


<br>

# `#04 Remote Calculator:`

<br>

<br>

# `#05 streaming client:`

<br>

In [ ]:

# ===  client.py === 
import socket
import subprocess
import threading
import time

HOST = 'localhost'
PORT = 9999
BUFFER_SIZE = 10000

client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

filename = input("Enter multimedia file to stream: ")
output_file = f"streaming_{filename}"
client.sendto(filename.encode(), (HOST, PORT))

response, _ = client.recvfrom(1024)
if response != b'OK':
    print("File not found on server")
    exit()

print("Streaming started...")
bytes_received = 0
player_launched = False

with open(output_file, 'wb') as f:
    while True:
        try:
            client.settimeout(2)
            data, _ = client.recvfrom(2048)
            
            if data == b'EOF':
                print("\nStreaming complete")
                break
            
            f.write(data)
            f.flush()
            bytes_received += len(data)
            print(f"Received: {bytes_received} bytes", end='\r')
            
            if bytes_received >= BUFFER_SIZE and not player_launched:
                print(f"\n{BUFFER_SIZE} bytes buffered. You can now play the file.")
                print(f"Run: vlc {output_file}  (or any media player)")
                player_launched = True
        
        except socket.timeout:
            print("\nStream timeout")
            break

print(f"File saved as: {output_file}")
client.close()


# === server.py ===
import socket
import random
import os
import time

server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server.bind(('localhost', 9999))
print(f"Streaming Server listening on {'localhost'}:{9999}")

while True:
    data, addr = server.recvfrom(1024)
    filename = data.decode()
    print(f"Client {addr} requested: {filename}")
    
    if os.path.exists(filename):
        server.sendto(b'OK', addr)
        time.sleep(0.1)
        
        with open(filename, 'rb') as f:
            while True:
                chunk_size = random.randint(1000, 2000)
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                server.sendto(chunk, addr)
                print(f"Sent {len(chunk)} bytes to {addr}")
                time.sleep(0.05)
        
        server.sendto(b'EOF', addr)
        print(f"Streaming complete for {filename}")
    else:
        server.sendto(b'ERROR', addr)
        print(f"File not found: {filename}")
        

<br>

# `#06 simple chating: `

<br>

In [ ]:

# === tcp_chat_client.py === 
import socket

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('localhost', 9999))
print(f"Connected to TCP Chat Server\n")
try:
    while True:
        msg = input("You: ")
        client.send(msg.encode())
        
        reply = client.recv(1024).decode()
        if not reply:
            break
        print(f"Server: {reply}")
except KeyboardInterrupt:
    print("\nChat ended")
client.close()


# === tcp_chat_server.py === 
import socket

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('localhost', 9999))
server.listen(1)

print(f"TCP Chat Server waiting on {'localhost'}:{9999}")
conn, addr = server.accept()
print(f"Connected to {addr}\n")

try:
    while True:
        msg = conn.recv(1024).decode()
        if not msg:
            break
        print(f"Client: {msg}")
        
        reply = input("You: ")
        conn.send(reply.encode())
except KeyboardInterrupt:
    print("\nChat ended")

conn.close()
server.close()


# === udp_chat_client.py === 
import socket
client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

print(f"Connected to UDP Chat Server\n")

try:
    while True:
        msg = input("You: ")
        client.sendto(msg.encode(), ('localhost', 9999))
        
        reply, _ = client.recvfrom(1000)
        print(f"Server: {reply.decode()}")
except KeyboardInterrupt:
    print("\nChat ended")

client.close()

# === udp_chat_server.py === 
import socket

server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server.bind(('localhost', 9999))

print(f"UDP Chat Server waiting on {'localhost'}:{9999}\n")

try:
    while True:
        msg, addr = server.recvfrom(1000)
        print(f"Client: {msg.decode()}")
        
        reply = input("You: ")
        server.sendto(reply.encode(), addr)
except KeyboardInterrupt:
    print("\nChat ended")

server.close()

<br>

# `#07 Multiple_client_single_server: `

<br>

In [ ]:

# === client.py ===
import socket
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('localhost', 9999))

print(f"Connected to Multi-Client Chat Server\n")

try:
    while True:
        msg = input("You: ")
        client.send(msg.encode())
        
        reply = client.recv(1024).decode()
        if not reply:
            break
        print(f"Server: {reply}")
except KeyboardInterrupt:
    print("\nChat ended")

client.close()

# === server.py ===
import socket
import threading
def handle_client(conn, addr):
    print(f"[NEW] Client {addr} connected")
    try:
        while True:
            msg = conn.recv(1024).decode()
            if not msg:
                break
            print(f"[{addr}] Client: {msg}")
            
            reply = input(f"[{addr}] You: ")
            conn.send(reply.encode())
    except:
        pass
    
    print(f"[DISCONNECTED] {addr}")
    conn.close()

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('localhost', 9999))
server.listen()

print(f"Multi-Client Chat Server on {'localhost'}:{9999}")
print("Waiting for clients...\n")

try:
    while True:
        conn, addr = server.accept()
        thread = threading.Thread(target=handle_client, args=(conn, addr))
        thread.start()
        print(f"[ACTIVE CONNECTIONS] {threading.active_count() - 1}\n")
except KeyboardInterrupt:
    print("\nServer stopped")
server.close()


<br>

# `#08 multicast_chat: `

<br>

In [ ]:
import socket
import threading
import struct

MCAST_GRP = '224.1.1.1'
MCAST_PORT = 5007

def receive_messages(sock):
    while True:
        try:
            data, addr = sock.recvfrom(1024)
            msg = data.decode()
            print(f"\n[{addr[0]}:{addr[1]}] {msg}")
            print("You: ", end='', flush=True)
        except:
            break

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(('', MCAST_PORT))

mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

print("Multicast Chat started")
print(f"Group: {MCAST_GRP}:{MCAST_PORT}\n")

receiver = threading.Thread(target=receive_messages, args=(sock,))
receiver.daemon = True
receiver.start()

try:
    while True:
        msg = input("You: ")
        sock.sendto(msg.encode(), (MCAST_GRP, MCAST_PORT))
except KeyboardInterrupt:
    print("\nChat ended")
sock.close()

<br>

# `#09 election voting : `

<br>

In [ ]:
import socket
import threading
import struct
import time

MCAST_GRP = '224.1.1.2'
MCAST_PORT = 5008
TOTAL_VOTERS = 5

votes = []
vote_lock = threading.Lock()

def receive_votes(sock):
    while True:
        try:
            data, addr = sock.recvfrom(1024)
            vote = data.decode()
            
            with vote_lock:
                if vote in ['A', 'B']:
                    votes.append(vote)
                    print(f"\nVote received: {vote} (Total: {len(votes)}/{TOTAL_VOTERS})")
                    
                    if len(votes) == TOTAL_VOTERS:
                        display_result()
                        break
        except:
            break

def display_result():
    count_a = votes.count('A')
    count_b = votes.count('B')
    
    print("\n" + "="*40)
    print("ELECTION RESULTS")
    print("="*40)
    print(f"Candidate A: {count_a} votes")
    print(f"Candidate B: {count_b} votes")
    
    if count_a > count_b:
        print("\nWINNER: Candidate A")
    elif count_b > count_a:
        print("\nWINNER: Candidate B")
    else:
        print("\nRESULT: TIE")
    print("="*40)

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(('', MCAST_PORT))

mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

print("Election Voting System")
print(f"Total Voters: {TOTAL_VOTERS}")
print("Candidates: A and B\n")

receiver = threading.Thread(target=receive_votes, args=(sock,))
receiver.daemon = True
receiver.start()

time.sleep(1)

while True:
    vote = input("Cast your vote (A or B): ").upper()
    if vote in ['A', 'B']:
        sock.sendto(vote.encode(), (MCAST_GRP, MCAST_PORT))
        print("Vote cast successfully!")
        break
    else:
        print("Invalid vote! Please enter A or B")

receiver.join()
sock.close()